![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/track-and-monitor-experiments/using-mlflow/train-projects-local/train-projects-local.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Train with MLflow Projects on local compute

Train MLflow Projects on your machine with local compute and AzureML tracking. In this notebook you will:

1. Set up MLflow tracking URI to track experiments and metrics in AzureML
2. Create experiment
3. Set up an MLflow project to run on AzureML compute
4. Submit an MLflow project run and view it in an AzureML workspace 

## Prerequisites 

If you are using a Notebook VM, you're all set. Otherwise, go through the [Configuration](../../../../configuration.ipnyb) notebook to set up your Azure Machine Learning workspace and ensure other common prerequisites are met.

Install azureml-mlflow package before running this notebook. Note that mlflow itself gets installed as dependency if you haven't installed it yet.

```
pip install azureml-mlflow
```

This example also uses scikit-learn. Install them using the following:
```
pip install scikit-learn matplotlib
```

Make sure you have the following before starting the notebook: 
- Connected to an AzureML Workspace 
- Your local conda environment has the necessary packages needed to run this project





## Set-up

Configure your workspace and check package versions

In [1]:
import sys, os
import mlflow
import mlflow.azureml

import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print("SDK version:", azureml.core.VERSION)
print("MLflow version:", mlflow.version.VERSION)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

SDK version: 1.39.0
MLflow version: 1.24.0
mlw-dev-geog-01
rg-dev-geog-eastus
eastus
062bbb35-45d7-40c6-937f-a43ab3667b0f


## Initialize Tracking Store and Experiment

### Set the Tracking Store
Set the MLflow tracking URI to point to your Azure ML Workspace. The subsequent logging calls from MLflow APIs will go to Azure ML services and will be tracked under your Workspace.

In [2]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

### Create Experiment
Create an Mlflow Experiment to organize your runs. It can be set either by passing the name as a parameter in the mlflow.projects.run call or by the following,

In [3]:
experiment_name = "train-project-local"
mlflow.set_experiment(experiment_name)

2022/04/04 15:52:09 INFO mlflow.tracking.fluent: Experiment with name 'train-project-local' does not exist. Creating a new experiment.


<Experiment: artifact_location='', experiment_id='fe3cdccc-dacb-41ad-99b8-f7e1d759b07a', lifecycle_stage='active', name='train-project-local', tags={}>

## Create the Backend Configuration Object

The backend configuration object will store necesary information for the integration such as the compute target and whether to use your local managed environment or a system managed environment. 

The integration will accept "COMPUTE" and "USE_CONDA" as parameters where "COMPUTE" is set to the name of a remote target (not applicable for this training example) and "USE_CONDA" which creates a new environment for the project from the environment configuration file. You must set this to "False" and include it in the backend configuration object if you want to use your local environment for the project run. Mlflow accepts a dictionary object or a JSON file.

In [4]:
# dictionary
backend_config = {"USE_CONDA": False}


## Add the Integration to your Environment Configuration

Add the azureml-mlflow package as a pip dependency to your environment configuration file (conda.yaml). The project can run without this addition, but key artifacts and metrics will not be logged to your Workspace. An example conda.yaml file is included in this notebook folder. Adding it to to the file will look like this,

```
name: mlflow-example
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.6
  - scikit-learn=0.19.1
  - pip
  - pip:
    - mlflow
    - azureml-mlflow
```

## User Managed environment
For using your local conda environment, set `use_conda = False` in the backend_config object. Ensure your local environment has all the necessary packages for running the project and you are specifying the **backend parameter** in any run call to be **"azureml"**.

In [7]:
local_env_run = mlflow.projects.run(uri=".", parameters={"alpha":0.3}, backend = "azureml", use_conda=False, backend_config = backend_config)


Class AzureMLProjectBackend: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022/04/04 15:54:02 INFO mlflow.projects.utils: === Created directory /tmp/tmpehs40wvm for downloading remote URIs passed to arguments of type 'path' ===


ExecutionException: Local conda environment not found, check your path.

In [8]:
local_env_run

NameError: name 'local_env_run' is not defined

Note: All these calculations were run on your local machine, in the conda environment you defined above. You can find the results in:
- Your AzureML Experiments (a link to the run will be provided in the console)
- ~/.azureml/envs/azureml_xxxx for the conda environment you just created
- ~/AppData/Local/Temp/azureml_runs/train-on-local_xxxx for the machine learning models you trained (this path may differ depending on the platform you use). This folder also contains
    - Logs (under azureml_logs/)
    - Output pickled files (under outputs/)
    - The configuration files (credentials, local and docker image setups)
    - The train.py and mylib.py scripts
    - The current notebook

## System Mananged Environment

Now, instead of managing the setup of the environment yourself, you can ask the system to build a new conda environment for you using the environment configuration file in this project. If a backend configuration object is not provided in the call, the integration will default to creating a new conda environment. The environment is built once, and will be reused in subsequent executions as long as the conda dependencies remain unchanged.


In [9]:
backend_config = {"USE_CONDA": True}
local_mlproject_run = mlflow.projects.run(uri=".", 
                                    parameters={"alpha":0.3},
                                    backend = "azureml",
                                    backend_config = backend_config)

Class AzureMLProjectBackend: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022/04/04 15:54:29 INFO mlflow.projects.utils: === Created directory /tmp/tmpsgq5db58 for downloading remote URIs passed to arguments of type 'path' ===
No Python version provided, defaulting to "3.6.2"
Class AzureMLSubmittedRun: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


04/04/2022 15:52:39 INFO azureml.mlflow: === Creating conda environment from Mlproject for local run ===
RunId: train-project-local_1649087673_cf63db18
Web View: https://ml.azure.com/runs/train-project-local_1649087673_cf63db18?wsid=/subscriptions/062bbb35-45d7-40c6-937f-a43ab3667b0f/resourcegroups/rg-dev-geog-eastus/workspaces/mlw-dev-geog-01&tid=5b6f6241-9a57-4be4-8e50-1dfa72e79a57

Execution Summary
RunId: train-project-local_1649087673_cf63db18
Web View: https://ml.azure.com/runs/train-project-local_1649087673_cf63db18?wsid=/subscriptions/062bbb35-45d7-40c6-937f-a43ab3667b0f/resourcegroups/rg-dev-geog-eastus/workspaces/mlw-dev-geog-01&tid=5b6f6241-9a57-4be4-8e50-1dfa72e79a57

Warnings:
{
  "error": {
    "code": "ServiceError",
    "severity": null,
    "message": "Failed to run ['/bin/bash', '/tmp/azureml_runs/train-project-local_1649087673_cf63db18/azureml-environment-setup/conda_env_builder.sh'] \n Exit code 1 \nDetails can be found in azureml-logs/60_control_log.txt log file.",

ExecutionException: Run (ID 'train-project-local_1649087673_cf63db18') failed

## Next Steps 

Try out these notebooks to learn more about MLflow-Azure Machine Learning integration:

 * [Train a model using remote compute on Azure Cloud](../train-on-remote/train-on-remote.ipynb)
 * [Deploy the model as a web service](../deploy-model/deploy-model.ipynb)
 * [Train a model using Pytorch and MLflow](../../ml-frameworks/using-mlflow/train-and-deploy-pytorch)

